In [ ]:
from manim import *
import numpy as np

config.media_width = "75%"
config.verbosity = "WARNING"

In [ ]:
# set parameters

# l_{p} norm
p = 2

# delta
d = 0.1

# number of chips
m = 2

# number of rounds
T = 3

# R^{n}
n = 2 # can't change

In [ ]:
# set weight and drift vectors

# weight vectors (w[i, t] = w_{i}^{t})
w = np.zeros([m+1, T+1, n]) # +1 to enable 1-indexing

w[1, 1] = [1, 1]
w[1, 2] = [1, 1]
w[1, 3] = [1, 1]

w[2, 1] = [-1, -1]
w[2, 2] = [-1, -1]
w[2, 3] = [-1, -1]


# # drift vectors (z[i, t] = z_{i}^{t})
z = np.zeros([m+1, T+1, n]) # +1 to enable 1-indexing

z[1, 1] = [1, 1]
z[1, 2] = [1, 1]
z[1, 3] = [1, 1]

z[2, 1] = [-1, -1]
z[2, 2] = [-1, -1]
z[2, 3] = [-1, -1]

In [ ]:
# check that the drift vectors are valid
for t in range(1, T+1):
    left = 0
    right = 0
    for i in range(1, m+1):
        left += np.dot(w[i, t], z[i, t])
        right += np.linalg.norm(w[i, t], p)
    right + d * right
    
    if left >= right:
        print("z_{{1}}^{{{b}}} - z_{{{a}}}^{{{b}}} are valid".format(a=n, b=t))
    else:
        print("z_{{1}}^{{{b}}} - z_{{{a}}}^{{{b}}} are not valid".format(a=n, b=t))

In [ ]:
class DriftingGame(Scene):
    def construct(self):
        
        # create R^2
        r2 = NumberPlane()
        self.play(Create(r2))
        
        self.wait()
        
        # create chips
        chips = [None] # None to enable 1 indexing
        for i in range(1, m+1):
            chips.append(
                Dot(
                    point = np.array([0, 0, 0]),
                    radius = 0.2,
                    color=ManimColor('#FFCB77')
                )
            )
        self.play(*[Create(chips[i]) for i in range(1, m+1)])
        
        self.wait()
        
        for t in range(1, 2): # range(1, T+1):
            
            # write round
            round = Text("Round {a}".format(a=t))
            round.scale(1.5)
            round.to_edge(UP)
            self.play(Write(round))
            
            self.wait()
            
            # create weight vectors
            weight_vec = [None] # None to enable 1 indexing
            for i in range(1, m+1):
                weight_vec.append(
                    LabeledArrow(
                        label = Tex(
                            "$w_{{{a}}}^{{{b}}}$".format(a=i, b=t)
                        ),
                        start = chips[i].get_center(),
                        end = chips[i].get_center() + np.append(w[i, t], 0),
                        buff = 0,
                        label_position = 1 + (0.5 / np.linalg.norm(w[i, t], 2)), 
                        label_frame = False,
                        color=ManimColor('#17C3B2')
                    )
                )
            self.play(*[GrowArrow(weight_vec[i]) for i in range(1, m+1)])

            self.wait()
            
            # remove weight vectors
            self.play(*[FadeOut(weight_vec[i]) for i in range(1, m+1)])

            self.wait()
            
            # create drift vectors
            drift_vec = [None] # None to enable 1 indexing
            for i in range(1, m+1):
                drift_vec.append(
                    LabeledArrow(
                        label = Tex(
                            "$z_{{{a}}}^{{{b}}}$".format(a=i, b=t)
                        ),
                        start = chips[i].get_center(),
                        end = chips[i].get_center() + np.append(z[i, t], 0),
                        buff = 0,
                        label_position = 1 + (0.5 / np.linalg.norm(z[i, t], 2)),
                        label_frame = False,
                        color=ManimColor('#FE6D73')
                    )
                )
            self.play(*[GrowArrow(drift_vec[i]) for i in range(1, m+1)])

            self.wait()
            
            # move chips
            for i in range(1, m+1):                
                chips[i].generate_target()
                chips[i].target.shift(np.append(z[i, t], 0))
            self.play(*[MoveToTarget(chips[i]) for i in range(1, m+1)])

            # remove drift vectors
            self.play(*[FadeOut(drift_vec[i]) for i in range(1, m+1)])
      
            self.wait()

            # erase round
            self.play(Unwrite(round))

            self.wait()

# don't remove below command for run button to work
%manim -qm -v WARNING DriftingGame